In [2]:
import chess
import numpy as np
import pandas as pd


In [3]:
def disp(fen):
    board = chess.Board(fen)
    return board


In [4]:
abs_path="d:\OneDrive - Indian Institute of Technology Bombay\Study\Events and Activities\Year 2\Seasons of Coding 23\soc23-DeepCarlsen"
TotalPos = pd.read_csv(abs_path+"\dataset\chessDataEval.csv")


In [5]:
def cond(arr):
    pos = arr["FEN"].split()[0]
    res = pos.count('Q')>1 or pos.count('B')>2 or pos.count('N')>2 or pos.count('R')>2 or pos.count('q')>1 or pos.count('b')>2 or pos.count('n')>2 or pos.count('r')>2
    return not res


In [7]:
dataset_raw = TotalPos[TotalPos.apply(cond, axis = 1)]


In [8]:
dataset1 = dataset_raw.sample(100000)

In [10]:
dataset = dataset1.sample(1000)
dataset.reset_index(inplace=True)
dataset = dataset.drop(["index"], axis=1)
dataset

,FEN,Evaluation
0,5r1k/1b4bp/p2p2p1/2p1P3/2BnP1q1/2N1B3/PP1Q2PP/...,0
1,1r3rk1/p3p1bp/3p1pp1/3P4/P1R1P3/1n2B3/5PPP/3N1...,-106
2,8/8/4n1K1/7P/1pkb4/8/8/3B4 w - - 3 54,-741
3,5bk1/pr3pp1/b2p4/P1pP2N1/2P1NR1P/8/4QPK1/1q6 b...,+1014
4,8/4prkp/3p2p1/p1pP4/PnB1P3/1P5P/5BP1/4R2K b - ...,0
...,...,...
995,r1bq1rk1/2p2ppp/p1n5/1pbnp3/8/2PP1N1P/PPB2PP1/...,0
996,8/p4p2/4pkp1/7p/2Pr1P1P/5KP1/P1R5/8 w - - 1 34,+63
997,7k/1p3R1p/p7/2p5/P1P1r1P1/7P/8/6K1 w - - 1 42,0
998,2r5/5p2/b1k1p1p1/1qPpP1Pp/3B1P2/pP2Q2P/P4K2/3R...,+62


In [11]:
def features_global(arr):
    fen = arr if isinstance(arr, str) else arr["FEN"]
    board = chess.Board(fen)
    white_play = (board.turn==chess.WHITE)
    K = board.has_kingside_castling_rights(chess.WHITE)
    k = board.has_kingside_castling_rights(chess.BLACK)
    Q = board.has_queenside_castling_rights(chess.WHITE)
    q = board.has_queenside_castling_rights(chess.BLACK)
    return [int(white_play),int(K), int(k), int(Q), int(q)]

In [12]:
def features_piece(arr):
    def p(symbol):
        return chess.Piece.from_symbol(symbol)
    fen = arr if isinstance(arr, str) else arr["FEN"]
    board = chess.Board(fen)
    pieces_map = chess.Board.piece_map(board)
    emp = [-1,-1]
    piece_num = [0]*14
    piece_w = [emp]*16  # K Q B(light) B(dark) N N R R P P P P P P P P
    piece_b = [emp]*16  # k q b(light) b(dark) n n r r p p p p p p p p
    for (loc, pi) in pieces_map.items():
        if pi== p('k'):
            piece_b[0] = [loc%8,loc//8]
            piece_num[7]+=1
        elif pi== p('q'):
            piece_b[1] = [loc%8,loc//8]
            piece_num[8]+=1
        elif pi== p('b'):
            if((loc%8+loc//8)%2!=0):
                piece_b[2] = [loc%8,loc//8]
                piece_num[9]+=1
            else:
                piece_b[3] = [loc%8,loc//8]
                piece_num[10]+=1
        elif pi== p('n'):
            piece_b[piece_b[4:6].index(emp)+4]=[loc%8,loc//8]
            piece_num[11]+=1
        elif pi== p('r'):
            piece_b[piece_b[6:8].index(emp)+6]=[loc%8,loc//8]
            piece_num[12]+=1
        elif pi== p('p'):
            piece_b[piece_b[8:16].index(emp)+8]=[loc%8,loc//8]
            piece_num[13]+=1

        elif pi== p('K'):
            piece_w[0] = [loc%8,loc//8]
            piece_num[0]+=1
        elif pi== p('Q'):
            piece_w[1] = [loc%8,loc//8]
            piece_num[1]+=1
        elif pi== p('B'):
            if((loc%8+loc//8)%2!=0):
                piece_w[2] = [loc%8,loc//8]
                piece_num[2]+=1
            else:
                piece_w[3] = [loc%8,loc//8]
                piece_num[3]+=1
        elif pi== p('N'):
            piece_w[piece_w[4:6].index(emp)+4]=[loc%8,loc//8]
            piece_num[4]+=1
        elif pi== p('R'):
            piece_w[piece_w[6:8].index(emp)+6]=[loc%8,loc//8]
            piece_num[5]+=1
        elif pi== p('P'):
            piece_w[piece_w[8:16].index(emp)+8]=[loc%8,loc//8]
            piece_num[6]+=1
    result = piece_num+[item for sublist in piece_w for item in sublist]+[item for sublist in piece_b for item in sublist]
    return result


In [13]:
def features_map(arr):
    fen = arr if isinstance(arr, str) else arr["FEN"]
    piece_value = {6:100, 5:9, 3:3, 2:3, 4:5, 1:1}
    board = chess.Board(fen)
    wht = []
    wht_count = []
    blk = []
    blk_count = []
    for sq in chess.SQUARES:   
        attks_white = chess.BaseBoard.attackers(board, chess.WHITE, sq)
        attks_black = chess.BaseBoard.attackers(board, chess.BLACK, sq)
        wht.append(0 if len(attks_white)==0 else min(piece_value[chess.BaseBoard.piece_type_at(board, sq2)] for sq2 in attks_white))
        blk.append(0 if len(attks_black)==0 else min(piece_value[chess.BaseBoard.piece_type_at(board, sq2)] for sq2 in attks_black))
        wht_count.append(len(attks_white))
        blk_count.append(len(attks_black))
    return wht_count+wht+blk_count+blk

In [8]:
arr = []
fen = "2q1rrk1/1b2ppbp/p2p1np1/P1pP4/2B1PP2/2NQ3P/1P1B2P1/R4RK1 w - - 5 22"
board = chess.Board(fen)
for sq in chess.SQUARES: 
    test = chess.BaseBoard.piece_type_at(board, sq)
    arr.append(0 if test==None else test)
arr

[4,
 0,
 0,
 0,
 0,
 4,
 6,
 0,
 0,
 1,
 0,
 3,
 0,
 0,
 1,
 0,
 0,
 0,
 2,
 5,
 0,
 0,
 0,
 1,
 0,
 0,
 3,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 2,
 1,
 0,
 0,
 3,
 0,
 0,
 1,
 1,
 3,
 1,
 0,
 0,
 5,
 0,
 4,
 4,
 6,
 0]

In [14]:
def feature_extract(evalPos):
        feat_centric = evalPos.copy().drop(["FEN", "Evaluation"], axis=1)
        feat_global = evalPos.copy().drop(["FEN", "Evaluation"], axis=1)
        col_centric = [ "cwk", "cwq", "cwbl","cwbd","cwn","cwr","cwp",
                "cbk", "cbq", "cbbl","cbbd","cbn","cbr","cbp", 
                'wk_col', 'wk_row', 'wq_col', 'wq_row', 'wbl_col', 'wbl_row', 'wbd_col', 'wbd_row', 'wn1_col', 'wn1_row', 'wn2_col', 'wn2_row', 'wr1_col', 'wr1_row', 'wr2_col', 'wr2_row',
                 'wp1_col', 'wp1_row', 'wp2_col', 'wp2_row', 'wp3_col', 'wp3_row', 'wp4_col', 'wp4_row', 'wp5_col', 'wp5_row', 'wp6_col', 'wp6_row', 'wp7_col', 'wp7_row', 'wp8_col', 'wp8_row',
                'bk_col', 'bk_row', 'bq_col', 'bq_row', 'bbl_col', 'bbl_row', 'bbd_col', 'bbd_row', 'bn1_col', 'bn1_row', 'bn2_col', 'bn2_row', 'br1_col', 'br1_row', 'br2_col', 'br2_row', 
                'bp1_col', 'bp1_row', 'bp2_col', 'bp2_row', 'bp3_col', 'bp3_row', 'bp4_col', 'bp4_row', 'bp5_col', 'bp5_row', 'bp6_col', 'bp6_row', 'bp7_col', 'bp7_row', 'bp8_col', 'bp8_row']
        col_global = ["white_turn", "castle_wk", "castle_bk", "castle_wq", "castle_bq"]
        col_transfer = ["cwk", "cwq", "cwbl","cwbd","cwn","cwr","cwp",
                        "cbk", "cbq", "cbbl","cbbd","cbn","cbr","cbp"]
        y = evalPos["Evaluation"]
        feat_centric[col_centric] = evalPos[["FEN"]].apply(features_piece, axis=1, result_type="expand")
        feat_global[col_global] = evalPos[["FEN"]].apply(features_global, axis=1, result_type="expand")
        feat_global = feat_global.join(feat_centric[col_transfer])
        feat_centric.drop(["cwk", "cwq", "cwbl","cwbd","cwn","cwr","cwp",
                "cbk", "cbq", "cbbl","cbbd","cbn","cbr","cbp"], axis=1, inplace=True)
        
        feat_map = evalPos.copy().drop(["FEN", "Evaluation"], axis=1)
        col_map = ["wht_count"+str(i) for i in range(64)]+["wht_attack"+str(i) for i in range(64)]+["blk_count"+str(i) for i in range(64)]+["blk_attack"+str(i) for i in range(64)]
        feat_map[col_map] = evalPos[["FEN"]].apply(features_map, axis=1, result_type="expand")
        feat_map
        return feat_global, feat_centric, feat_map, y

In [15]:
col_centric = [ "cwk", "cwq", "cwbl","cwbd","cwn","cwr","cwp",
                "cbk", "cbq", "cbbl","cbbd","cbn","cbr","cbp", 
                'wk_col', 'wk_row', 'wq_col', 'wq_row', 'wbl_col', 'wbl_row', 'wbd_col', 'wbd_row', 'wn1_col', 'wn1_row', 'wn2_col', 'wn2_row', 'wr1_col', 'wr1_row', 'wr2_col', 'wr2_row',
                 'wp1_col', 'wp1_row', 'wp2_col', 'wp2_row', 'wp3_col', 'wp3_row', 'wp4_col', 'wp4_row', 'wp5_col', 'wp5_row', 'wp6_col', 'wp6_row', 'wp7_col', 'wp7_row', 'wp8_col', 'wp8_row',
                'bk_col', 'bk_row', 'bq_col', 'bq_row', 'bbl_col', 'bbl_row', 'bbd_col', 'bbd_row', 'bn1_col', 'bn1_row', 'bn2_col', 'bn2_row', 'br1_col', 'br1_row', 'br2_col', 'br2_row', 
                'bp1_col', 'bp1_row', 'bp2_col', 'bp2_row', 'bp3_col', 'bp3_row', 'bp4_col', 'bp4_row', 'bp5_col', 'bp5_row', 'bp6_col', 'bp6_row', 'bp7_col', 'bp7_row', 'bp8_col', 'bp8_row']
print(len(col_centric))
fen = "2q1rrk1/1b2ppbp/p2p1np1/P1pP4/2B1PP2/2NQ3P/1P1B2P1/R4RK1 w - - 5 22"
board = chess.Board(fen)
ar = features_piece(fen)
print(len(ar))

78
78


In [16]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.layers import Dense

In [17]:
from tensorflow.keras.optimizers import Adam
import math
from tensorflow.keras import backend as K
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [18]:
feat1, feat2, feat3, y_raw = feature_extract(dataset)

C:\Users\spand\AppData\Local\Temp\ipykernel_38092\1613683386.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feat_map[col_map] = evalPos[["FEN"]].apply(features_map, axis=1, result_type="expand")
C:\Users\spand\AppData\Local\Temp\ipykernel_38092\1613683386.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feat_map[col_map] = evalPos[["FEN"]].apply(features_map, axis=1, result_type="expand")
C:\Users\spand\AppData\Local\Temp\ipykernel_38092\1613683386.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usu

In [37]:
x = pd.concat([feat1, feat2, feat3], axis=1)
y = y_raw.apply(lambda l: int(l) if '#' not in l else +9999999 if '+' in l else -9999999)
x.describe()

,white_turn,castle_wk,castle_bk,castle_wq,castle_bq,cwk,cwq,cwbl,cwbd,cwn,...,blk_attack54,blk_attack55,blk_attack56,blk_attack57,blk_attack58,blk_attack59,blk_attack60,blk_attack61,blk_attack62,blk_attack63
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.0,1000.000000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.516000,0.052000,0.052000,0.043000,0.047000,1.0,0.615000,0.474000,0.458000,0.841000,...,46.129000,40.896000,3.912000,5.376000,6.159000,8.273000,9.646000,17.716000,15.687000,36.552000
std,0.499994,0.222138,0.222138,0.202959,0.211745,0.0,0.486839,0.499573,0.498482,0.761774,...,48.412769,47.724132,11.363952,14.412715,16.395809,20.967501,23.757548,34.380421,32.390032,46.622236
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,...,3.000000,0.000000,0.000000,0.000000,0.000000,3.000000,3.000000,3.000000,3.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.000000,0.000000,0.000000,1.000000,...,9.000000,5.000000,3.000000,4.000000,5.000000,5.000000,3.000000,3.000000,5.000000,5.000000
75%,1.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.000000,1.000000,1.000000,1.000000,...,100.000000,100.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,100.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,2.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000


In [ ]:
tf.convert_to_tensor(np.asarray(x).astype(np.float_))

C:\Users\spand\AppData\Local\Temp\ipykernel_24644\159561658.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  tf.convert_to_tensor(np.asarray(x).astype(np.float))


<tf.Tensor: shape=(1000, 339), dtype=float64, numpy=
array([[  1.,   0.,   0., ...,   3.,   5., 100.],
       [  1.,   0.,   0., ...,   5.,   5., 100.],
       [  0.,   0.,   0., ...,   0., 100.,   0.],
       ...,
       [  1.,   0.,   0., ...,   3.,   9.,   9.],
       [  1.,   0.,   0., ...,   3.,   0., 100.],
       [  1.,   0.,   0., ...,   5.,   5., 100.]])>

In [20]:
def tens(arra):
    return tf.convert_to_tensor(np.asarray(arra).astype(np.float_))

In [38]:
model = tf.keras.Sequential([Dense(512, activation="sigmoid"), Dense(128, activation="relu"), Dense(32, activation="relu"), Dense(1)])
model.compile(loss=rmse, optimizer=Adam(), metrics=[rmse])


In [39]:
nlim = int(len(x)*.8)
model.fit(tens(x[:nlim]), tens(y[:nlim]), epochs=10 )

Epoch 1/10
25/25 [==============================] - 1s 7ms/step - loss: 636828.2500 - rmse: 636828.2500
Epoch 2/10
25/25 [==============================] - 0s 5ms/step - loss: 547237.0625 - rmse: 547237.0625
Epoch 3/10
25/25 [==============================] - 0s 4ms/step - loss: 595484.9375 - rmse: 595484.9375
Epoch 4/10
25/25 [==============================] - 0s 4ms/step - loss: 554110.6875 - rmse: 554110.6875
Epoch 5/10
25/25 [==============================] - 0s 4ms/step - loss: 505872.9062 - rmse: 505872.9062
Epoch 6/10
25/25 [==============================] - 0s 4ms/step - loss: 554018.1875 - rmse: 554018.1875
Epoch 7/10
25/25 [==============================] - 0s 4ms/step - loss: 636921.0625 - rmse: 636921.0625
Epoch 8/10
25/25 [==============================] - 0s 4ms/step - loss: 595381.5625 - rmse: 595381.5625
Epoch 9/10
25/25 [==============================] - 0s 4ms/step - loss: 636782.3750 - rmse: 636782.3750
Epoch 10/10
25/25 [==============================] - 0s 4ms/step

In [35]:
y_pred = model.predict(tens(x[nlim:]))
rmse(y_pred.flatten(), y[nlim:])

7/7 [==============================] - 0s 3ms/step


<tf.Tensor: shape=(), dtype=float64, numpy=649.1954331133768>

In [36]:
df = pd.DataFrame({'Actual': y[nlim:], 'Predicted': y_pred.flatten()})
df

,Actual,Predicted
800,0,280.920105
801,-999,263.876190
802,1639,224.780167
803,241,188.757462
804,7,325.475464
...,...,...
995,0,52.673889
996,63,-212.265244
997,0,261.850616
998,62,314.862549
